In [1]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [2]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    #water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na*qe*1e10/(1000*eps_0)
    water_dipole = 6.17e-30/(1e-10)/((2*Rw)**2) * Na*qe*1e10/(1000*eps_0)
    print water_dipole
    print sigma_s[:5]*1.4
    print max(abs(sigma_s*1.4))
    '''
    test_charge = numpy.where(abs(sigma_d)*1.4>water_dipole)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
    '''    
    
    test_charge = numpy.where(abs(sigma_s)*1.4>water_dipole)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy, float(len(test_charge))/N_stern

In [12]:
phi_static = 10. #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 6.2

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(problem_folder+namefile))
energy = numpy.zeros(num_lines)
test_charge_arr = numpy.zeros(num_lines)
it = 0
for line in file(problem_folder+namefile):
    mol = line[:-1]

    mesh_stern = problem_folder+mol+'/surf_d02_stern_fix'
    mesh_diel = problem_folder+mol+'/surf_d02_fix'

    try: 
        e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
    except:
        print 'Something went wrong'
        e = 0
        test_charge = 0.
            
    ekJ = e*conv_factor # kJ/mol
    ekcal = ekJ/4.184   # kcal/mol
    
    energy[it] = ekcal
    test_charge_arr[it] = test_charge
    
    print 'Run %i for '%it + mol + '. Energy = %1.3f kcal/mol'%energy[it]
    
    it += 1
    
numpy.savetxt('energy_bem.txt', energy)
numpy.savetxt('test_charge.txt', test_charge_arr)

857.593881087
[  105.11879687 -1974.27345599  -587.89268656  -766.17312015  -742.88331056]
16777.4626311
Unphysical charge in stern surface
91
Run 0 for 1112_tetrachloroethane. Energy = 10.787 kcal/mol
857.593881087
[ -668.93666189 -2272.88186941 -1084.95109788  -699.50521535  -834.00851965]
19193.2055036
Unphysical charge in stern surface
86
Run 1 for 111_trichloroethane. Energy = 9.969 kcal/mol
857.593881087
[  30.51404595 -408.98213937 -140.61110608  189.19032023  113.92733956]
26568.9967379
Unphysical charge in stern surface
137
Run 2 for 111_trifluoro_222_trimethoxyethane. Energy = 12.886 kcal/mol
857.593881087
[ -435.89958412 -2272.29115968 -1089.87706536  -777.06115621  -894.9778927 ]
15806.0166191
Unphysical charge in stern surface
87
Run 3 for 111_trifluoropropan_2_ol. Energy = 9.761 kcal/mol
857.593881087
[ 146.36474196 -122.84141145 -132.34314506  166.40208311  -46.87146226]
32448.7039826
Unphysical charge in stern surface
127
Run 4 for 111_trimethoxyethane. Energy = 12.090 

857.593881087
[ -157.06608133 -2250.61273014 -2286.60049986  -229.59129595  -458.19018524]
14335.2221148
Unphysical charge in stern surface
151
Run 42 for 1_bromohexane. Energy = 13.850 kcal/mol
857.593881087
[-400.37394933 -604.83824403 -768.83404317 -145.32392238 -316.29862461]
17545.9050646
Unphysical charge in stern surface
183
Run 43 for 1_bromooctane. Energy = 16.594 kcal/mol
857.593881087
[-374.41729071 -605.63642521 -450.22220937 -477.74479998 -486.54440103]
15635.57847
Unphysical charge in stern surface
116
Run 44 for 1_bromopentane. Energy = 12.442 kcal/mol
857.593881087
[-351.68290309 -353.22809889 -218.0615779  -329.28871757 -172.29567501]
15359.5807862
Unphysical charge in stern surface
79
Run 45 for 1_bromopropane. Energy = 9.662 kcal/mol
857.593881087
[ -538.74717907 -1727.50653507  -708.41308965  -620.09384762  -918.02675344]
15388.9193731
Unphysical charge in stern surface
50
Run 46 for 1_chloro_222_trifluoroethane. Energy = 9.089 kcal/mol
857.593881087
[ -275.19092378

857.593881087
[  -3.3530391  -119.72356947   16.00257995  254.69088894 -452.25599674]
34839.9859252
Unphysical charge in stern surface
132
Run 84 for 25_dimethylphenol. Energy = 12.754 kcal/mol
857.593881087
[-328.08762625 -603.53249889 -554.66499246 -368.82735361 -553.94489315]
14876.9581115
Unphysical charge in stern surface
132
Run 85 for 25_dimethylpyridine. Energy = 12.142 kcal/mol
857.593881087
[-525.16440968 -499.34068163 -608.21071945 -260.34760848 -294.36536747]
17721.5457465
Unphysical charge in stern surface
135
Run 86 for 25_dimethyltetrahydrofuran. Energy = 11.800 kcal/mol
857.593881087
[ -39.44660622   -2.02559393  -10.14177223 -154.09251317 -104.4260573 ]
23687.3122162
Unphysical charge in stern surface
165
Run 87 for 26_dimethylaniline. Energy = 12.771 kcal/mol
857.593881087
[ -380.23552125  -529.7747489   -237.23580368  -189.13770028 -1151.95041361]
17638.012121
Unphysical charge in stern surface
248
Run 88 for 26_dimethylnaphthalene. Energy = 15.449 kcal/mol
857.59388

857.593881087
[  215.21792812 -1823.04230642  -793.0313385   -649.66689692  -950.8746016 ]
15144.547453
Unphysical charge in stern surface
88
Run 126 for 2_methylpropan_1_ol. Energy = 9.869 kcal/mol
857.593881087
[ -369.27142655 -1665.90828695  -819.42421422  -851.42585961  -847.83725653]
14804.9306769
Unphysical charge in stern surface
95
Run 127 for 2_methylpropan_2_ol. Energy = 9.877 kcal/mol
857.593881087
[ -714.92387165 -1032.30978819  -712.50990091  -825.93447563 -1929.6523153 ]
16075.3517044
Unphysical charge in stern surface
92
Run 128 for 2_methylpropane. Energy = 9.561 kcal/mol
857.593881087
[ -864.67593688 -1738.23348698  -702.52292011   445.33230109  -746.09496646]
28677.2908698
Unphysical charge in stern surface
126
Run 129 for 2_methylpropene. Energy = 9.223 kcal/mol
857.593881087
[-327.29533962 -731.35939356 -507.54161884 -470.25795996 -632.53777821]
47304.9082215
Unphysical charge in stern surface
104
Run 130 for 2_methylpyrazine. Energy = 10.328 kcal/mol
857.593881087


857.593881087
[ -749.65677912  -984.4497794   -654.34618825  -888.92955738 -1737.4328628 ]
16100.7459274
Unphysical charge in stern surface
163
Run 168 for 3_methylheptane. Energy = 14.669 kcal/mol
857.593881087
[-257.59506247 -362.80557431 -372.84792405 -463.56026911 -476.04484307]
592890.211892
Unphysical charge in stern surface
154
Run 169 for 3_methylhexane. Energy = 13.080 kcal/mol
857.593881087
[-161.70617339 -636.23783472 -210.10940213 -717.47370063 -517.67828339]
15064.6444419
Unphysical charge in stern surface
110
Run 170 for 3_methylpentane. Energy = 11.902 kcal/mol
857.593881087
[ 513.16524224 -600.86741676 -195.73786853  164.86168636 -113.47013835]
15403.5126158
Unphysical charge in stern surface
106
Run 171 for 3_methylpyridine. Energy = 10.662 kcal/mol
857.593881087
[ -638.46983535 -1198.00055078  -783.17332866  -879.94951097  -792.07773078]
87973.2708589
Unphysical charge in stern surface
96
Run 172 for 3_nitroaniline. Energy = 11.909 kcal/mol
857.593881087
[ -886.462306

...

857.593881087
[ -313.34148204   394.7322469   -545.82894191 -1146.90683468  -724.5113755 ]
26989.4575376
Unphysical charge in stern surface
168
Run 296 for diisopropylamine. Energy = 12.707 kcal/mol
857.593881087
[ 199.48373794 -770.35868366 -103.47466018  574.25079884   -5.5588635 ]
21540.8450285
Unphysical charge in stern surface
82
Run 297 for dimethoxymethane. Energy = 9.089 kcal/mol
857.593881087
[-968.54107477 -945.3575012  -818.48387241  -69.38974002 -970.83369225]
42781.0700307
Unphysical charge in stern surface
78
Run 298 for dimethyl_disulfide. Energy = 9.540 kcal/mol
857.593881087
[-549.4316687  -536.23588403 -458.63114796 -630.7627522  -960.55558884]
17793.6242653
Unphysical charge in stern surface
39
Run 299 for dimethyl_ether. Energy = 7.288 kcal/mol
857.593881087
[ 223.26580949  -43.84890729   84.57618913   81.00167061   89.65314787]
18334.9646888
Unphysical charge in stern surface
108
Run 300 for dimethyl_sulfate. Energy = 10.650 kcal/mol
857.593881087
[  27.95388821 -8

857.593881087
[ -752.71408062 -1004.20806714  -799.00199147  -689.98781778  -757.18852379]
13796.5931249
Unphysical charge in stern surface
139
Run 339 for hexanoic_acid. Energy = 13.141 kcal/mol
857.593881087
[-297.77839    -293.72217745   13.78319572   72.56493644 -290.69606421]
377.379913793
Run 340 for hydrazine. Energy = 5.115 kcal/mol
Something went wrong
Run 341 for hydrogen_sulfide. Energy = 0.000 kcal/mol
857.593881087
[-316.49677484 -615.08441581 -541.62380661 -673.13132839 -712.40224452]
17895.1933497
Unphysical charge in stern surface
29
Run 342 for imidazole. Energy = 7.866 kcal/mol
857.593881087
[ -583.16691284 -1640.60138761  -863.87912168  -954.27905448  -704.9557795 ]
63651.0470168
Unphysical charge in stern surface
184
Run 343 for indane. Energy = 12.571 kcal/mol
857.593881087
[-889.91776189 -717.43928025 -949.11715056 -505.6422431  -862.66985962]
24068.5458698
Unphysical charge in stern surface
155
Run 344 for iodobenzene. Energy = 11.495 kcal/mol
857.593881087
[-351

857.593881087
[-793.28428338 -957.7677987  -658.28441913 -587.56295855 -913.6094492 ]
21828.4225251
Unphysical charge in stern surface
168
Run 382 for methyl_p_methoxybenzoate. Energy = 14.827 kcal/mol
857.593881087
[ -873.57446361 -1046.54698845  -340.57168569 -1135.50120432  -504.69579757]
61231.4615707
Unphysical charge in stern surface
154
Run 383 for methyl_p_nitrobenzoate. Energy = 14.643 kcal/mol
857.593881087
[-9470.93880115  7488.64749447  -653.50568503 -1226.49214169  -673.17219155]
38924.8370234
Unphysical charge in stern surface
150
Run 384 for methyl_pentanoate. Energy = 13.300 kcal/mol
857.593881087
[ 751.29541915 -553.6957965   368.67093169 -270.96096825  200.7855065 ]
20657.3433877
Unphysical charge in stern surface
110
Run 385 for methyl_propanoate. Energy = 10.507 kcal/mol
857.593881087
[ -569.84449731  -919.25049015  -731.18190341  -771.56110383 -1561.99476238]
22263.0619761
Unphysical charge in stern surface
116
Run 386 for methyl_propyl_ether. Energy = 10.395 kcal/

857.593881087
[ -531.02750016 -1870.19273879  -518.77983662  -628.86907503  -546.69978508]
22477.1655608
Unphysical charge in stern surface
150
Run 424 for n_propylcyclopentane. Energy = 13.620 kcal/mol
857.593881087
[ 1229.40165251  -636.6294123     41.52899755  -176.2686192    113.76315109]
30964.7798347
Unphysical charge in stern surface
193
Run 425 for naphthalene. Energy = 12.744 kcal/mol
857.593881087
[-905.10126659 -690.57306193 -816.86274947 -600.74520784 -979.29187678]
92730.1849505
Unphysical charge in stern surface
108
Run 426 for nitrobenzene. Energy = 11.284 kcal/mol
857.593881087
[-286.13904871 -551.00356405 -336.62975213 -127.29174441 -294.60234437]
30228.0700863
Unphysical charge in stern surface
71
Run 427 for nitroethane. Energy = 8.568 kcal/mol
857.593881087
[ 257.4817701  -141.3660807  -170.75096462   60.27691891 -128.74590332]
101970.561283
Unphysical charge in stern surface
31
Run 428 for nitromethane. Energy = 7.090 kcal/mol
857.593881087
[-516.56978269 -429.1277

857.593881087
[ -567.4515044  -1378.81075632  -654.47483709  -757.84733028  -605.92566072]
15503.0947508
Unphysical charge in stern surface
97
Run 467 for propan_2_ol. Energy = 8.739 kcal/mol
857.593881087
[-436.48313448 -553.62556892 -427.60300136 -479.94614934 -449.75831495]
28026.1165221
Unphysical charge in stern surface
69
Run 468 for propane. Energy = 8.392 kcal/mol
857.593881087
[ -298.87315818   468.40130913  -807.4425335  -1105.20529735  -693.74703174]
18303.5293112
Unphysical charge in stern surface
63
Run 469 for propanenitrile. Energy = 8.555 kcal/mol
857.593881087
[-112.01983484 -239.06668485 -396.40563654  167.3489474   123.01789875]
19405.9268322
Unphysical charge in stern surface
74
Run 470 for propanoic_acid. Energy = 8.862 kcal/mol
857.593881087
[ 213.5694665  -100.41660578 -276.72287433   79.03686634 -487.2245515 ]
14678.7383004
Unphysical charge in stern surface
88
Run 471 for propanone. Energy = 8.479 kcal/mol
857.593881087
[  665.87333013   950.07534171   152.8424

In [12]:
mesh_stern = 'mobley_test/nitroethane/surf_d02_stern'
mesh_diel = 'mobley_test/nitroethane/surf_d02'

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 5.5#80.

phi_static = 46. #kJ/mol/e
conv_factor = (1000*eps_0)/(qe**2*Na*1e10)


e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e * conv_factor#/ ((qe**2*Na*1e10)/(1000*eps_0))
ekcal = ekj/4.184
#print e, e / ((qe**2*Na*1e10)/(1000*eps_0)), e/(1000*4.184) #kJ/mol
print ekcal

857.593881087
[-278.6805626  -496.77562928 -318.64588094 -114.92787056 -278.07517599]
28648.1780082
Unphysical charge in stern surface
45
8.82394356584


In [33]:
grid_diel = generate_grid(mesh_diel)
grid_stern = generate_grid(mesh_stern)

space_diel = bempp.api.function_space(grid_diel, "DP", 0)
space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
N_diel = grid_diel.leaf_view.entity_count(0)
N_stern = grid_stern.leaf_view.entity_count(0)
    
elements_d = list(grid_diel.leaf_view.entity_iterator(0))
elements_s = list(grid_stern.leaf_view.entity_iterator(0))

print elements_d[0].geometry.corners
print elements_d[0].geometry.local2global(numpy.array([[1.,0.,0.]]))

[[-0.802 -0.885 -0.46 ]
 [ 1.119  0.807  1.406]
 [ 0.339  0.758  0.614]]


AttributeError: 'Geometry' object has no attribute 'global2local'

In [20]:
F = 1000*eps_0/(qe**2*Na*1e10)
1/(4*numpy.pi*F*4.184)
9081457.48434/(332.063*4*numpy.pi)
(qe**2*Na*1e10)/(eps_0)
2353.39653991/40

58.834913497749994

In [19]:
# Check (1/2+V)phi_1 is zero to avoid induced charge

def compute_induced_charge_term(mesh):
    
    grid = generate_grid(mesh)
    space = bempp.api.function_space(grid, "DP", 0)
    
    N = grid.leaf_view.entity_count(0)
    phi_static = 10.
    phi_static *= 4.184
    
    dirichlet_fun = bempp.api.GridFunction(space, coefficients=numpy.ones(N)*phi_static)

    identity = sparse.identity(space, space, space)
    dlp      = laplace.double_layer(space, space, space)

    induced_charge_term = (.5 * identity + dlp) * dirichlet_fun
    
    return numpy.max(numpy.abs(induced_charge_term.coefficients))
    
    
#phi_static = 10. #kcal/mol/e
#phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 6.2

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(namefile))
induced_charge_term_array = numpy.zeros(num_lines)
it = 0
for line in file(namefile):
    
    mol = line[:-1]
    mesh = problem_folder+mol+'/surf_d02_fix'
    print 'Run %i for '%it + mol 
    
    try: 
        induced_charge_term = compute_induced_charge_term(mesh)
    except:
        print 'Something went wrong'
        induced_charge_term = 0

    
    induced_charge_term_array[it] = induced_charge_term    
    
    it += 1
    
print numpy.max(induced_charge_term_array)

Run 0 for 1112_tetrachloroethane
Run 1 for 111_trichloroethane
Run 2 for 111_trifluoro_222_trimethoxyethane
Run 3 for 111_trifluoropropan_2_ol
Run 4 for 111_trimethoxyethane
Run 5 for 1122_tetrachloroethane
Run 6 for 112_trichloro_122_trifluoroethane
Run 7 for 112_trichloroethane
Run 8 for 11_diacetoxyethane
Run 9 for 11_dichloroethane
Run 10 for 11_dichloroethene
Run 11 for 11_diethoxyethane
Run 12 for 11_difluoroethane
Run 13 for 1234_tetrachlorobenzene
Run 14 for 1235_tetrachlorobenzene
Run 15 for 123_trichlorobenzene
Run 16 for 123_trimethylbenzene
Run 17 for 1245_tetrachlorobenzene
Run 18 for 124_trichlorobenzene
Run 19 for 124_trimethylbenzene
Run 20 for 12_diacetoxyethane
Run 21 for 12_dibromoethane
Run 22 for 12_dichlorobenzene
Run 23 for 12_dichloroethane
Run 24 for 12_dichloropropane
Run 25 for 12_diethoxyethane
Run 26 for 12_dimethoxyethane
Run 27 for 12_ethanediol
Run 28 for 135_trichlorobenzene
Run 29 for 135_trimethylbenzene
Run 30 for 13_dichlorobenzene
Run 31 for 13_dic

Run 280 for di_n_butyl_ether
Run 281 for di_n_butylamine
Run 282 for di_n_propyl_ether
Run 283 for di_n_propyl_sulfide
Run 284 for di_n_propylamine
Run 285 for dibromomethane
Run 286 for dichloromethane
Run 287 for diethoxymethoxybenzene
Run 288 for diethyl_disulfide
Run 289 for diethyl_ether
Run 290 for diethyl_malonate
Run 291 for diethyl_succinate
Run 292 for diethyl_sulfide
Run 293 for diethylamine
Run 294 for diiodomethane
Run 295 for diisopropyl_ether
Run 296 for diisopropylamine
Run 297 for dimethoxymethane
Run 298 for dimethyl_disulfide
Run 299 for dimethyl_ether
Run 300 for dimethyl_sulfate
Run 301 for dimethyl_sulfide
Run 302 for dimethyl_sulfone
Run 303 for dimethyl_sulfoxide
Run 304 for dimethylamine
Run 305 for ethanamide
Run 306 for ethane
Run 307 for ethanethiol
Run 308 for ethanol
Run 309 for ethene
Run 310 for ethyl_acetate
Run 311 for ethyl_benzoate
Run 312 for ethyl_butanoate
Run 313 for ethyl_formate
Run 314 for ethyl_hexanoate
Run 315 for ethyl_pentanoate
Run 316 f